In [1]:
from config.config import SQLQuery
import pandas as pd

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

import scipy.stats as stats
from scipy.stats import skew  
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from sklearn.metrics import roc_auc_score, recall_score, precision_score
# from imblearn.over_sampling import SMOTE

In [3]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import model_selection
kfolds = model_selection.KFold(n_splits=3, shuffle=True, random_state=50)

import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
import warnings
warnings.filterwarnings("ignore")

# Classifier
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, recall_score, precision_score,f1_score, accuracy_score, confusion_matrix
# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold

# Defining Evaluation Metrics

In [5]:
def evaluate(y_train, y_pred_train, y_test, y_pred_test):
    print('Test Recall', recall_score(y_test, y_pred_test,pos_label=1))
    print('Test Precision', precision_score(y_test, y_pred_test,pos_label=1))
    print('Test F1', f1_score(y_test, y_pred_test,pos_label=1))
    print('Test ROC_AUC',roc_auc_score(y_test, y_pred_test))
    print('Test Accuracy',accuracy_score(y_test, y_pred_test))
    print('Test Confusion Matrix',confusion_matrix(y_test, y_pred_test))
    ConfusionMatrix = confusion_matrix(y_test, y_pred_test)
    defaulters = ConfusionMatrix[1,0]
    success = ConfusionMatrix[0,0]
    default_rate = (ConfusionMatrix[1,0]) / (ConfusionMatrix[1,0] + ConfusionMatrix[0,0])
    print('Test Default Rate : ', default_rate)
    ax = sns.heatmap(ConfusionMatrix, annot=True, cmap='BuPu',fmt=".1f",cbar=False)
    ax.set_title('Confusion Matrix for Test');
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Actual');
    ax.xaxis.set_ticklabels(['Completed','Return'])
    ax.yaxis.set_ticklabels(['Completed','Return'])
    plt.show()
    
    print()
    
    print('Train Recall', recall_score(y_train, y_pred_train,pos_label=1))
    print('Train Precision', precision_score(y_train, y_pred_train,pos_label=1))
    print('Train F1', f1_score(y_train, y_pred_train,pos_label=1))
    print('Train ROC_AUC',roc_auc_score(y_train, y_pred_train))
    print('Train Accuracy',accuracy_score(y_train, y_pred_train))
    print('Train Confusion Matrix',confusion_matrix(y_train, y_pred_train))
    ConfusionMatrix = confusion_matrix(y_train, y_pred_train)
    defaulters = ConfusionMatrix[1,0]
    success = ConfusionMatrix[0,0]
    default_rate = (ConfusionMatrix[1,0]) / (ConfusionMatrix[1,0] + ConfusionMatrix[0,0])
    print('Train Default Rate : ', default_rate)
    ax = sns.heatmap(ConfusionMatrix, annot=True, fmt=".1f", cmap='BuPu',cbar=False)
    ax.set_title('Confusion Matrix for Train');
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Actual');
    ax.xaxis.set_ticklabels(['Completed','Return'])
    ax.yaxis.set_ticklabels(['Completed','Return'])
    plt.show()

In [ ]:
#Read data
import pandas as pd
x_train = pd.read_csv('/Users/saliljain/Downloads/x_train_less_6_months1.csv')
y_train = pd.read_csv('/Users/saliljain/Downloads/y_train_less_6_months.csv')

x_test = pd.read_csv('/Users/saliljain/Downloads/x_test_less_6_months.csv')
y_test = pd.read_csv('/Users/saliljain/Downloads/y_test_less_6_months.csv')

x_oot = pd.read_csv('/Users/saliljain/Downloads/pfr_less_x.csv')
y_oot = pd.read_csv('/Users/saliljain/Downloads/pfr_less_y.csv')

In [ ]:
x_oot.head()

In [26]:
# hyperparameters["iterations"] = "400"
# hyperparameters["eval_metric"] = "AUC"
# hyperparameters["depth"] = "6"
# hyperparameters["l2_leaf_reg"] = "70"
# hyperparameters["learning_rate"] = "0.11"
# hyperparameters["random_state"] = "42"
# hyperparameters["cat_features"] = ['bank_risk']
# hyperparameters["early_stopping_rounds"] = "700"


In [27]:

categoricals = ['bank_risk']

cb = CatBoostClassifier(cat_features=categoricals,
                         eval_metric = 'AUC',
                         depth = 6,
                         iterations = 400,
                         l2_leaf_reg = 70,
                         learning_rate = 0.11,
                        random_state = 42,
                        use_best_model=True
                       )

cb.fit(x_train, y_train, verbose = 1, eval_set = (x_test, y_test))

y_train_pred = cb.predict_proba(x_train)
y_test_pred = cb.predict_proba(x_test)

0:	test: 0.6286356	best: 0.6286356 (0)	total: 19.3ms	remaining: 7.69s
1:	test: 0.6384149	best: 0.6384149 (1)	total: 40.9ms	remaining: 8.14s
2:	test: 0.6544590	best: 0.6544590 (2)	total: 57.8ms	remaining: 7.65s
3:	test: 0.6577171	best: 0.6577171 (3)	total: 72.5ms	remaining: 7.18s
4:	test: 0.7513382	best: 0.7513382 (4)	total: 87.8ms	remaining: 6.93s
5:	test: 0.7765478	best: 0.7765478 (5)	total: 99.2ms	remaining: 6.51s
6:	test: 0.7857794	best: 0.7857794 (6)	total: 114ms	remaining: 6.4s
7:	test: 0.7874514	best: 0.7874514 (7)	total: 128ms	remaining: 6.27s
8:	test: 0.7923082	best: 0.7923082 (8)	total: 141ms	remaining: 6.14s
9:	test: 0.7925655	best: 0.7925655 (9)	total: 156ms	remaining: 6.09s
10:	test: 0.7961004	best: 0.7961004 (10)	total: 168ms	remaining: 5.93s
11:	test: 0.7997128	best: 0.7997128 (11)	total: 180ms	remaining: 5.83s
12:	test: 0.8011784	best: 0.8011784 (12)	total: 192ms	remaining: 5.71s
13:	test: 0.8011898	best: 0.8011898 (13)	total: 205ms	remaining: 5.64s
14:	test: 0.8030918	b

130:	test: 0.8332377	best: 0.8333658 (104)	total: 1.66s	remaining: 3.41s
131:	test: 0.8331791	best: 0.8333658 (104)	total: 1.67s	remaining: 3.4s
132:	test: 0.8331817	best: 0.8333658 (104)	total: 1.69s	remaining: 3.39s
133:	test: 0.8331932	best: 0.8333658 (104)	total: 1.7s	remaining: 3.37s
134:	test: 0.8331924	best: 0.8333658 (104)	total: 1.71s	remaining: 3.35s
135:	test: 0.8331890	best: 0.8333658 (104)	total: 1.72s	remaining: 3.34s
136:	test: 0.8331896	best: 0.8333658 (104)	total: 1.73s	remaining: 3.32s
137:	test: 0.8331890	best: 0.8333658 (104)	total: 1.74s	remaining: 3.31s
138:	test: 0.8331950	best: 0.8333658 (104)	total: 1.75s	remaining: 3.29s
139:	test: 0.8331614	best: 0.8333658 (104)	total: 1.76s	remaining: 3.28s
140:	test: 0.8331974	best: 0.8333658 (104)	total: 1.78s	remaining: 3.27s
141:	test: 0.8332000	best: 0.8333658 (104)	total: 1.79s	remaining: 3.25s
142:	test: 0.8332011	best: 0.8333658 (104)	total: 1.8s	remaining: 3.24s
143:	test: 0.8332021	best: 0.8333658 (104)	total: 1.81

255:	test: 0.8347601	best: 0.8347922 (241)	total: 3.11s	remaining: 1.75s
256:	test: 0.8347606	best: 0.8347922 (241)	total: 3.12s	remaining: 1.74s
257:	test: 0.8347535	best: 0.8347922 (241)	total: 3.13s	remaining: 1.72s
258:	test: 0.8347299	best: 0.8347922 (241)	total: 3.14s	remaining: 1.71s
259:	test: 0.8347533	best: 0.8347922 (241)	total: 3.15s	remaining: 1.7s
260:	test: 0.8347530	best: 0.8347922 (241)	total: 3.16s	remaining: 1.68s
261:	test: 0.8347415	best: 0.8347922 (241)	total: 3.17s	remaining: 1.67s
262:	test: 0.8347984	best: 0.8347984 (262)	total: 3.19s	remaining: 1.66s
263:	test: 0.8348737	best: 0.8348737 (263)	total: 3.2s	remaining: 1.65s
264:	test: 0.8348207	best: 0.8348737 (263)	total: 3.21s	remaining: 1.64s
265:	test: 0.8348068	best: 0.8348737 (263)	total: 3.22s	remaining: 1.62s
266:	test: 0.8347356	best: 0.8348737 (263)	total: 3.23s	remaining: 1.61s
267:	test: 0.8347279	best: 0.8348737 (263)	total: 3.25s	remaining: 1.6s
268:	test: 0.8347791	best: 0.8348737 (263)	total: 3.26

375:	test: 0.8357813	best: 0.8359376 (372)	total: 4.54s	remaining: 290ms
376:	test: 0.8357711	best: 0.8359376 (372)	total: 4.55s	remaining: 278ms
377:	test: 0.8356775	best: 0.8359376 (372)	total: 4.57s	remaining: 266ms
378:	test: 0.8356693	best: 0.8359376 (372)	total: 4.58s	remaining: 254ms
379:	test: 0.8356595	best: 0.8359376 (372)	total: 4.59s	remaining: 242ms
380:	test: 0.8356702	best: 0.8359376 (372)	total: 4.6s	remaining: 229ms
381:	test: 0.8356772	best: 0.8359376 (372)	total: 4.61s	remaining: 217ms
382:	test: 0.8357742	best: 0.8359376 (372)	total: 4.62s	remaining: 205ms
383:	test: 0.8356807	best: 0.8359376 (372)	total: 4.63s	remaining: 193ms
384:	test: 0.8357232	best: 0.8359376 (372)	total: 4.65s	remaining: 181ms
385:	test: 0.8357647	best: 0.8359376 (372)	total: 4.66s	remaining: 169ms
386:	test: 0.8357779	best: 0.8359376 (372)	total: 4.67s	remaining: 157ms
387:	test: 0.8357304	best: 0.8359376 (372)	total: 4.68s	remaining: 145ms
388:	test: 0.8357304	best: 0.8359376 (372)	total: 4.

# Feature Importance

In [29]:
print(cb.get_feature_importance())
arr_imp_cb = cb.get_feature_importance()
pd.DataFrame(arr_imp_cb)


[12.23919133 24.71769039  4.33486385  0.49527722  3.04054834  0.5550531
  4.92191363  1.96498864  2.75561023  2.48453832  7.92170735  2.5021986
  1.14634655  4.83377813  3.39517384  3.02794217  1.6707648   1.4341179
  0.48017176  2.53765609  5.97525503  7.56521273]


,0
0,12.239191
1,24.717690
2,4.334864
3,0.495277
4,3.040548
5,0.555053
6,4.921914
7,1.964989
8,2.755610
9,2.484538


# SHAP Testing

In [16]:
import shap
from catboost import CatBoostClassifier, Pool
shap.initjs()

In [19]:
explainer = shap.TreeExplainer(cb)
shap_values = explainer.shap_values(Pool(x_train, y_train, cat_features=categoricals))

filter_x_oot = x_oot[x_oot.index == 0]
filter_y_oot = y_oot[y_oot.index == 0]
filter_x_oot.head()

In [21]:
shap_values = explainer.shap_values(Pool(filter_x_oot, filter_y_oot, cat_features=categoricals))
shap.force_plot(explainer.expected_value, shap_values[0,:], filter_x_oot.iloc[0,:])